# Data Cleanup

## Setup

### Import Packages

In [195]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from collections import OrderedDict
from datetime import datetime, date
from os import environ
import json

# Set ipython's max row display
pd.set_option('display.max_row', 1000)
# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = [10, 5]

### Global Variables

## Importing Dataset

### Combined dataset

In [311]:
read_name = "../data/bronze_tables/combined_dataset.csv"

df_combined = pd.read_csv(read_name)

C:\Users\abdul\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [312]:
df_combined.head()

,Unnamed: 0,Entity_x,Code,Year,Annual CO2 emissions,Entity_y,Fossil fuels (% growth),Entity_x.1,Annual change in primary energy consumption (%),Entity_y.1,Gas Consumption - TWh,Coal Consumption - TWh,Oil Consumption - TWh,Entity_x.2,Fossil fuels (TWh),Entity_y.2,Coal Production - TWh,Oil Production - TWh,Gas Production - TWh,Entity_x.3,Fossil fuels per capita (kWh),Entity_y.3,Fossil fuels (% equivalent primary energy),Entity_x.4,"Gas (TWh, direct energy)","Oil (TWh, direct energy)","Coal (TWh, direct energy)",Entity_y.4,Geo Biomass Other - TWh,Solar Generation - TWh,Wind Generation - TWh,Hydro Generation - TWh,Entity_x.5,prod of Electricity from wind (TWh),prod of Electricity from hydro (TWh),prod of Electricity from solar (TWh),prod of Other renewables including bioenergy (TWh),Entity_y.5,Per capita electricity (kWh),Entity_x.6,Renewables per capita (kWh - equivalent),Entity_y.6,Renewables (% electricity)
0,0,Afghanistan,AFG,1949,14656.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Afghanistan,AFG,1950,84272.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Afghanistan,AFG,1951,91600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Afghanistan,AFG,1952,91600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Afghanistan,AFG,1953,106256.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [313]:
df_combined = df_combined[df_combined.columns[1:]]

In [314]:
df = df_combined.copy()

#### Unify Entity Columns

In [317]:
entity_columns = []
for col in df.columns:
    if "Entity" in col:
        entity_columns.append(col)

In [318]:
df_entities = df[entity_columns + ['Code']].drop_duplicates(entity_columns)

In [319]:
df['Entity'] = df['Entity_x']

for col in entity_columns:
    df['Entity'] = df['Entity'].fillna(df[col])

In [320]:
df['Entity'].isna().sum()

0

In [321]:
df = df.drop(columns=entity_columns)

In [322]:
df['Entity'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan',
       'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus',
       'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia',
       'Bonaire Sint Eustatius and Saba', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei',
       'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon',
       'Canada', 'Cape Verde', 'Central African Republic', 'Chad',
       'Chile', 'China', 'Christmas Island', 'Colombia', 'Comoros',
       'Congo', 'Cook Islands', 'Costa Rica', "Cote d'Ivoire", 'Croatia',
       'Cuba', 'Curacao', 'Cyprus', 'Czechia',
       'Democratic Republic of Congo', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiop

#### Drop Duplicate Columns

In [270]:
cols = list(df.columns)
unique_cols = {}

for col in cols:
    unique_cols[col] = col.split('_', 1)[0]

In [271]:
df = df.rename(columns = unique_cols)

In [272]:
df = df.loc[:,~df.columns.duplicated()]

ordered_columns = ['Entity', *df.columns[:-1]]
df = df[ordered_columns]

In [273]:
df.head()

,Entity,Code,Year,Annual CO2 emissions,Fossil fuels (% growth),Annual change in primary energy consumption (%),Gas Consumption - TWh,Coal Consumption - TWh,Oil Consumption - TWh,Fossil fuels (TWh),Coal Production - TWh,Oil Production - TWh,Gas Production - TWh,Fossil fuels per capita (kWh),Fossil fuels (% equivalent primary energy),"Gas (TWh, direct energy)","Oil (TWh, direct energy)","Coal (TWh, direct energy)",Geo Biomass Other - TWh,Solar Generation - TWh,Wind Generation - TWh,Hydro Generation - TWh,prod of Electricity from wind (TWh),prod of Electricity from hydro (TWh),prod of Electricity from solar (TWh),prod of Other renewables including bioenergy (TWh),Per capita electricity (kWh),Renewables per capita (kWh - equivalent),Renewables (% electricity)
0,Afghanistan,AFG,1949,14656.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1950,84272.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1951,91600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1952,91600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1953,106256.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Income Level Lookup table

In [274]:
read_name = "../data/bronze_tables/income_level_lookup.xlsx"

df_income_lookup = pd.read_excel(read_name, skiprows=0)

df_income_lookup.head()

,Economy,Code,Region,Income group,Lending category,Other (EMU or HIPC)
0,Aruba,ABW,Latin America & Caribbean,High income,NaN,NaN
1,Afghanistan,AFG,South Asia,Low income,IDA,HIPC
2,Angola,AGO,Sub-Saharan Africa,Lower middle income,IBRD,NaN
3,Albania,ALB,Europe & Central Asia,Upper middle income,IBRD,NaN
4,Andorra,AND,Europe & Central Asia,High income,NaN,NaN


In [275]:
df_income_lookup = df_income_lookup[df_income_lookup.columns[:5]]

#### join with combined_df

In [276]:
# df_income_lookup = df_income_lookup.rename(columns={'Economy': 'Entity'})
df = df.set_index('Code').join(df_income_lookup.set_index('Code')).reset_index()

### Population & Area

In [277]:
read_name = "../data/bronze_tables/API_EN.POP.DNST_DS2_en_csv_v2_4701323.csv"

df_population = pd.read_csv(read_name, skiprows=4)

df_population.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,...,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,Unnamed: 66
0,Aruba,ABW,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,307.966667,312.411111,314.994444,316.827778,318.650000,320.566667,322.466667,324.316667,326.300000,328.166667,330.233333,332.494444,334.644444,336.261111,336.961111,336.588889,335.366667,333.900000,333.177778,333.872222,...,484.888889,494.494444,504.811111,516.066667,527.733333,538.977778,548.577778,555.711111,560.166667,562.366667,563.122222,563.622222,564.805556,566.944444,569.805556,573.138889,576.533333,579.661111,582.583333,585.338889,588.033333,590.611111,593.144444,NaN,NaN
1,Africa Eastern and Southern,AFE,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,9.206929,9.444024,9.690214,9.945378,10.209576,10.482895,10.765896,11.059617,11.365371,11.684171,12.016528,12.362473,12.721761,13.094008,13.478945,13.876693,14.287579,14.711372,15.148660,15.599232,...,25.947604,26.625607,27.321141,28.035481,28.769895,29.526360,30.001659,30.801413,31.628679,32.483559,33.367205,34.279582,35.219737,36.189915,36.879528,37.898051,38.940522,40.004465,41.089451,42.195162,43.319881,44.462045,45.620592,NaN,NaN
2,Afghanistan,AFG,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,14.058547,14.337645,14.631648,14.940699,15.265041,15.600080,15.945197,16.308762,16.702347,17.131463,17.594177,18.078319,18.565480,19.031569,19.455045,19.844369,20.194247,20.454746,20.561857,20.478206,...,30.261978,30.925972,31.859861,33.127872,34.651540,36.307546,37.910996,39.333171,40.527204,41.550591,42.503842,43.534959,44.747269,46.176059,47.776671,49.475786,51.164166,52.762987,54.249311,55.649251,56.992046,58.325678,59.684990,NaN,NaN
3,Africa Western and Central,AFW,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,10.877837,11.109940,11.351399,11.601453,11.859717,12.126330,12.402029,12.687791,12.984878,13.294421,13.616671,13.952068,14.302085,14.669124,15.053820,15.456176,15.876543,16.313802,16.766255,17.232596,...,28.002263,28.761516,29.540244,30.338334,31.157344,32.000505,32.872024,33.774930,34.710713,35.678698,36.677028,37.702723,38.753638,39.829118,40.929916,42.056948,43.211639,44.394917,45.607045,46.847251,48.114080,49.405535,50.720207,NaN,NaN
4,Angola,AGO,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,4.436874,4.498676,4.555554,4.600140,4.628678,4.637286,4.631780,4.629801,4.655231,4.724761,4.845784,5.012405,5.211585,5.423617,5.634069,5.839119,6.043005,6.249117,6.463553,6.690695,...,12.320205,12.727096,13.151101,13.592487,14.052633,14.535555,15.046232,15.588036,16.162593,16.768557,17.402451,18.059096,18.734457,19.427817,20.139508,20.867720,21.610470,22.366552,23.135062,23.916555,24.713072,25.527632,26.362612,NaN,NaN


In [278]:
read_name = "../data/bronze_tables/API_AG.LND.TOTL.K2_DS2_en_csv_v2_4701206.csv"

df_area = pd.read_csv(read_name, skiprows=4)

df_area.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,...,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,Unnamed: 66
0,Aruba,ABW,Land area (sq. km),AG.LND.TOTL.K2,NaN,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,...,180.0,180.0,180.0,180.0,180.0,180.0,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,180.00,1.800000e+02,1.800000e+02,180.00,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,NaN,NaN
1,Africa Eastern and Southern,AFE,Land area (sq. km),AG.LND.TOTL.K2,NaN,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,...,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,1.472019e+07,1.472024e+07,1.472023e+07,1.472027e+07,14720236.89,1.472027e+07,1.472096e+07,14721240.05,1.484517e+07,1.484513e+07,1.484509e+07,1.484514e+07,1.484515e+07,1.484514e+07,1.484515e+07,1.484516e+07,1.484512e+07,NaN,NaN
2,Afghanistan,AFG,Land area (sq. km),AG.LND.TOTL.K2,NaN,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,...,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,6.522300e+05,6.522300e+05,6.522300e+05,6.522300e+05,652230.00,6.522300e+05,6.522300e+05,652230.00,6.522300e+05,6.522300e+05,6.522300e+05,6.522300e+05,6.522300e+05,6.522300e+05,6.522300e+05,6.522300e+05,6.522300e+05,NaN,NaN
3,Africa Western and Central,AFW,Land area (sq. km),AG.LND.TOTL.K2,NaN,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046180.0,9045780.0,9045780.0,9045780.0,9045780.0,9045780.0,9045780.0,...,9045780.0,9045780.0,9045780.0,9045780.0,9045780.0,9045780.0,9.045780e+06,9.045780e+06,9.045780e+06,9.045780e+06,9045780.00,9.045780e+06,9.045780e+06,9045780.00,9.045780e+06,9.045780e+06,9.045780e+06,9.045780e+06,9.045780e+06,9.045780e+06,9.045780e+06,9.045780e+06,9.045773e+06,NaN,NaN
4,Angola,AGO,Land area (sq. km),AG.LND.TOTL.K2,NaN,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,...,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1.246700e+06,1.246700e+06,1.246700e+06,1.246700e+06,1246700.00,1.246700e+06,1.246700e+06,1246700.00,1.246700e+06,1.246700e+06,1.246700e+06,1.246700e+06,1.246700e+06,1.246700e+06,1.246700e+06,1.246700e+06,1.246700e+06,NaN,NaN


In [279]:
df_pop_area = pd.concat([df_population, df_area])

df_pop_area = df_pop_area[df_population.columns[:-2]].drop(['Indicator Code'], axis=1)

In [280]:
df_pop_area.head(2)

,Country Name,Country Code,Indicator Name,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,...,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,ABW,Population density (people per sq. km of land ...,NaN,307.966667,312.411111,314.994444,316.827778,318.650000,320.566667,322.466667,324.316667,326.300000,328.166667,330.233333,332.494444,334.644444,336.261111,336.961111,336.588889,335.366667,333.900000,333.177778,333.872222,336.450000,...,462.283333,474.722222,484.888889,494.494444,504.811111,516.066667,527.733333,538.977778,548.577778,555.711111,560.166667,562.366667,563.122222,563.622222,564.805556,566.944444,569.805556,573.138889,576.533333,579.661111,582.583333,585.338889,588.033333,590.611111,593.144444
1,Africa Eastern and Southern,AFE,Population density (people per sq. km of land ...,NaN,9.206929,9.444024,9.690214,9.945378,10.209576,10.482895,10.765896,11.059617,11.365371,11.684171,12.016528,12.362473,12.721761,13.094008,13.478945,13.876693,14.287579,14.711372,15.148660,15.599232,16.062631,...,24.633762,25.284822,25.947604,26.625607,27.321141,28.035481,28.769895,29.526360,30.001659,30.801413,31.628679,32.483559,33.367205,34.279582,35.219737,36.189915,36.879528,37.898051,38.940522,40.004465,41.089451,42.195162,43.319881,44.462045,45.620592


In [281]:
df_pop_area = df_pop_area.melt(
    id_vars=df_pop_area.columns[:3], 
    value_vars=df_pop_area.columns[3:],
    var_name='Year'
).reset_index(drop=True)

df_pop_area.head()

,Country Name,Country Code,Indicator Name,Year,value
0,Aruba,ABW,Population density (people per sq. km of land ...,1960,NaN
1,Africa Eastern and Southern,AFE,Population density (people per sq. km of land ...,1960,NaN
2,Afghanistan,AFG,Population density (people per sq. km of land ...,1960,NaN
3,Africa Western and Central,AFW,Population density (people per sq. km of land ...,1960,NaN
4,Angola,AGO,Population density (people per sq. km of land ...,1960,NaN


In [282]:
df_pop_area = df_pop_area.pivot(['Country Code', 'Year'], 'Indicator Name', 'value').reset_index()

In [283]:
df_pop_area['Year'] = df_pop_area['Year'].astype(np.number)

#### join with combined_df

In [284]:
df_pop_area = df_pop_area.rename(columns={'Country Name': 'Entity'})

In [285]:
df_pop_area.head()

Indicator Name,Country Code,Year,Land area (sq. km),Population density (people per sq. km of land area)
0,ABW,1960.0,NaN,NaN
1,ABW,1961.0,180.0,307.966667
2,ABW,1962.0,180.0,312.411111
3,ABW,1963.0,180.0,314.994444
4,ABW,1964.0,180.0,316.827778


In [286]:
df = df.set_index(['Code', 'Year']).join(df_pop_area.set_index(['Country Code', 'Year'])).reset_index()

### Rearrange Columns

In [288]:
cat_cols = ['Entity', 'Year', 'Code', 'Region', 'Income group', 'Lending category']
value_cols = list((set(df.columns) - set(cat_cols)) - {'Country Code'})

In [289]:
df = df[cat_cols + value_cols]

### Basic Analysis

In [291]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3548764 entries, 0 to 3548763
Data columns (total 35 columns):
 #   Column                                               Dtype  
---  ------                                               -----  
 0   Entity                                               object 
 1   Year                                                 int64  
 2   Code                                                 object 
 3   Region                                               object 
 4   Income group                                         object 
 5   Lending category                                     object 
 6   Renewables (% electricity)                           float64
 7   Coal (TWh, direct energy)                            float64
 8   Land area (sq. km)                                   float64
 9   Economy                                              object 
 10  Renewables per capita (kWh - equivalent)             float64
 11  Fossil fuels (% equivale

In [292]:
df.describe()

,Year,Renewables (% electricity),"Coal (TWh, direct energy)",Land area (sq. km),Renewables per capita (kWh - equivalent),Fossil fuels (% equivalent primary energy),Fossil fuels per capita (kWh),Annual CO2 emissions,Geo Biomass Other - TWh,Coal Consumption - TWh,Wind Generation - TWh,prod of Electricity from solar (TWh),Per capita electricity (kWh),Coal Production - TWh,Gas Production - TWh,Fossil fuels (TWh),Solar Generation - TWh,Oil Production - TWh,prod of Other renewables including bioenergy (TWh),Population density (people per sq. km of land area),"Gas (TWh, direct energy)","Oil (TWh, direct energy)",prod of Electricity from hydro (TWh),Fossil fuels (% growth),Annual change in primary energy consumption (%),Gas Consumption - TWh,Hydro Generation - TWh,prod of Electricity from wind (TWh),Oil Consumption - TWh
count,3.548764e+06,1.481185e+06,15179.000000,3.413246e+06,1.122327e+06,1.115945e+06,1.115945e+06,3.424429e+06,1.103980e+06,1.115948e+06,1.103980e+06,1.866353e+06,1.524543e+06,362327.000000,655474.000000,1.115945e+06,1.103980e+06,665848.000000,1.849595e+06,3.191901e+06,15179.000000,15179.000000,1.884703e+06,1.110365e+06,2.466964e+06,1.122331e+06,1.122327e+06,1.866353e+06,1.122334e+06
mean,1.990835e+03,2.955008e+01,28051.366649,5.249864e+06,4.521244e+00,8.641344e+01,3.256567e+04,2.143474e+08,5.271232e+00,7.492411e+02,5.355192e+00,1.071659e+00,3.889834e+03,1896.652969,927.879854,2.324555e+03,1.801836e+00,1702.987533,3.196876e+00,2.917047e+02,20841.881473,38418.751094,3.734946e+01,3.518550e+00,4.529973e+00,5.562669e+02,6.128578e+01,3.180940e+00,1.011397e+03
std,1.794527e+01,3.208910e+01,9862.128852,1.509725e+07,1.334260e+01,1.555028e+01,3.447909e+04,1.742079e+09,3.296575e+01,3.740826e+03,5.427216e+01,1.794764e+01,5.077855e+03,6091.733428,3556.924536,1.096832e+04,2.330776e+01,5833.765701,2.559484e+01,1.503785e+03,9417.580614,9569.777345,2.362517e+02,2.724270e+01,3.188959e+01,2.738557e+03,3.037947e+02,4.182296e+01,4.682507e+03
min,1.750000e+03,0.000000e+00,97.000000,2.027000e+00,0.000000e+00,1.280470e+01,1.558074e+02,3.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.339596,0.000000,9.738055e-02,0.000000e+00,0.000000,0.000000e+00,9.862452e-02,0.000000,0.000000,0.000000e+00,-4.959083e+01,-9.500508e+01,0.000000e+00,0.000000e+00,0.000000e+00,9.738056e-02
25%,1.976000e+03,1.307330e+00,19457.904297,2.164000e+04,1.683346e-01,8.080704e+01,1.152530e+04,5.935680e+05,0.000000e+00,3.265833e+00,0.000000e+00,0.000000e+00,5.642858e+02,37.866039,45.318611,1.164674e+02,0.000000e+00,102.932060,0.000000e+00,1.956125e+01,13293.952148,33660.101562,1.400000e-02,-1.060772e+00,-8.340657e-01,8.645000e+00,4.206000e-01,0.000000e+00,6.454946e+01
50%,1.992000e+03,1.599574e+01,25904.468750,1.998100e+05,7.866168e-01,9.176839e+01,2.540453e+04,4.968423e+06,3.900000e-02,3.254101e+01,0.000000e+00,0.000000e+00,2.427437e+03,125.336014,140.884018,2.823127e+02,0.000000e+00,344.388245,0.000000e+00,5.159331e+01,20063.484375,37960.613281,1.520000e+00,2.466643e+00,2.337921e+00,5.259425e+01,4.582918e+00,0.000000e+00,1.425319e+02
75%,2.006000e+03,5.312500e+01,38069.035156,1.260000e+06,2.784231e+00,9.784651e+01,4.025504e+04,4.217100e+07,1.299000e+00,1.681976e+02,5.770781e-02,1.094000e-03,5.367686e+03,824.191345,399.114014,9.501196e+02,4.572000e-03,1135.137939,3.500000e-01,1.286045e+02,28161.105469,46672.656250,9.815488e+00,6.335747e+00,6.949067e+00,2.325982e+02,2.291953e+01,5.000000e-03,4.147270e+02
max,2.021000e+03,1.000000e+02,45161.207031,1.299870e+08,1.538834e+02,1.000000e+02,3.087042e+05,3.670250e+10,7.627827e+02,4.516121e+04,1.861940e+03,1.032501e+03,5.678160e+04,46550.605469,40368.828125,1.361315e+05,1.032501e+03,52181.949219,7.627827e+02,2.138860e+04,40374.605469,53368.628906,4.345990e+03,1.553105e+03,1.553105e+03,4.037461e+04,4.345990e+03,1.861940e+03,5.336863e+04


In [293]:
df.describe(include='object')

,Entity,Code,Region,Income group,Lending category,Economy
count,3548764,3548764,3283267,3266991,2248104,3283267
unique,235,235,7,4,3,211
top,World,GBR,Europe & Central Asia,High income,IBRD,United Kingdom
freq,16437,16437,838218,1155284,1119762,16437


### Data Clean-up

In [294]:
x = 'Entity'

#### Unify Entinty Names

In [295]:
df[x] = df[x].str.title()

#### Create Entity Category Column

In [296]:
df[x].nunique()

235

In [309]:
df[x].unique()

array(['United Kingdom', 'World', 'Canada', 'Germany', 'Poland',
       'United States', 'Belgium', 'France', 'Austria', 'Norway',
       'Armenia', 'Azerbaijan', 'Belarus', 'Spain', 'Estonia', 'Georgia',
       'Hungary', 'Kazakhstan', 'Kyrgyzstan', 'Lithuania', 'Latvia',
       'Moldova', 'Russia', 'Tajikistan', 'Turkmenistan', 'Ukraine',
       'Uzbekistan', 'Sweden', 'Denmark', 'Netherlands', 'Ireland',
       'Switzerland', 'India', 'Romania', 'Australia', 'Czechia',
       'Finland', 'Italy', 'Slovakia', 'Turkey', 'Greece', 'Japan',
       'Portugal', 'New Zealand', 'Bulgaria', 'Peru', 'South Africa',
       'Bosnia And Herzegovina', 'Croatia', 'North Macedonia',
       'Montenegro', 'Serbia', 'Slovenia', 'Argentina', 'Indonesia',
       'Malaysia', 'Mexico', 'Vietnam', 'Chile', 'Taiwan', 'China',
       'Brazil', 'Zimbabwe', 'Venezuela', 'South Korea', 'North Korea',
       'Iran', 'Philippines', 'Trinidad And Tobago', 'Egypt', 'Nigeria',
       'Algeria', 'Tunisia', 'Ecuador', 

In [297]:
continents = ['Asia', 'Africa', 'North America', 'South America', 'Europe', "Antarctica"]

countries_official_names = ['Afghanistan', 'Aland Islands', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia, Plurinational State of', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Congo', 'Congo, The Democratic Republic of the', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Falkland Islands (Malvinas)', 'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia', 'French Southern Territories', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'Grenada', 'Guadeloupe', 'Guam', 'Guatemala', 'Guernsey', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Heard Island and McDonald Islands', 'Holy See (Vatican City State)', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Republic of', 'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jersey', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', "Korea, Democratic People's Republic of", 'Korea, Republic of', 'Kuwait', 'Kyrgyzstan', "Lao People's Democratic Republic", 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macao', 'Macedonia, Republic of', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Martinique', 'Mauritania', 'Mauritius', 'Mayotte', 'Mexico', 'Micronesia, Federated States of', 'Moldova, Republic of', 'Monaco', 'Mongolia', 'Montenegro', 'Montserrat', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Caledonia', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'Niue', 'Norfolk Island', 'Northern Mariana Islands', 'Norway', 'Oman', 'Pakistan', 'Palau', 'Palestinian Territory, Occupied', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Pitcairn', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar', 'Réunion', 'Romania', 'Russian Federation', 'Rwanda', 'Saint Barthélemy', 'Saint Helena, Ascension and Tristan da Cunha', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Martin (French part)', 'Saint Pierre and Miquelon', 'Saint Vincent and the Grenadines', 'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore', 'Sint Maarten (Dutch part)', 'Slovakia', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 'South Georgia and the South Sandwich Islands', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'South Sudan', 'Svalbard and Jan Mayen', 'Swaziland', 'Sweden', 'Switzerland', 'Syrian Arab Republic', 'Taiwan, Province of China', 'Tajikistan', 'Tanzania, United Republic of', 'Thailand', 'Timor-Leste', 'Togo', 'Tokelau', 'Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Turkmenistan', 'Turks and Caicos Islands', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 'United States Minor Outlying Islands', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela, Bolivarian Republic of', 'Viet Nam', 'Virgin Islands, British', 'Virgin Islands, U.S.', 'Wallis and Futuna', 'Yemen', 'Zambia', 'Zimbabwe']
other_countries = [
       'Antigua And Barbuda', 'Bolivia',
       'Bonaire Sint Eustatius And Saba', 'Bosnia And Herzegovina',
       'British Virgin Islands', 'Brunei', "Cote D'Ivoire", 'Curacao',
       'Czechia', 'Democratic Republic Of Congo', 'Eswatini',
       'Faeroe Islands', 'French Equatorial Africa', 'French West Africa',
       'Iran',
       'Kosovo',  'Laos', 'Leeward Islands',
       'Moldova',
       'North Korea', 'North Macedonia', 'Oceania', 'Palestine',
       'Panama Canal Zone', 'Reunion', 'Russia', 'Ryukyu Islands',
       'Saint Helena', 'Saint Kitts And Nevis',
       'Saint Pierre And Miquelon', 'Saint Vincent And The Grenadines',
       'Sao Tome And Principe', 'South Korea', 'St. Kitts-Nevis-Anguilla',
       'Syria', 'Taiwan', 'Tanzania', 'Timor', 'Trinidad And Tobago',
       'Turks And Caicos Islands',
       'Venezuela', 'Vietnam', 'Wallis And Futuna', 'Ussr',
       'Czechoslovakia', 'Falkland Islands', 'Netherlands Antilles',
       'Serbia And Montenegro', 'United States Virgin Islands',
       'Western Sahara', 'Yugoslavia',
       'Micronesia (Country)', 'Sint Maarten (Dutch Part)'
                  ]
countries = countries_official_names + other_countries

In [298]:
df[x].isna().sum()

0

In [299]:
def assign_category_to_entity(s):

    if "(" in s:
        if "(Bp)" in s:
            return "Bp?"
        if "(Eia)" in s:
            return "Eia?"
        if "(Ember)" in s:
            return "Ember?"
        if s in countries:
            return "Country"
        else:
            return "Exlusive Regions"

    else:
        if s == "World":
            return "World"
        elif s in continents:
            return "Continent"
        elif s in countries:
            return "Country"
        elif "Income" in s:
            return "Income Level"
        else:
            return "Other"

In [300]:
df['Entity_Category'] = df[x].apply(assign_category_to_entity)

In [301]:
df['Entity_Category'].value_counts()

Country      3521953
World          16437
Continent      10374
Name: Entity_Category, dtype: int64

In [310]:
df.loc[df['Entity_Category'] == 'Continent']

,Entity,Year,Code,Region,Income group,Lending category,Renewables (% electricity),"Coal (TWh, direct energy)",Land area (sq. km),Economy,Renewables per capita (kWh - equivalent),Fossil fuels (% equivalent primary energy),Fossil fuels per capita (kWh),Annual CO2 emissions,Geo Biomass Other - TWh,Coal Consumption - TWh,Wind Generation - TWh,prod of Electricity from solar (TWh),Per capita electricity (kWh),Coal Production - TWh,Gas Production - TWh,Fossil fuels (TWh),Solar Generation - TWh,Oil Production - TWh,prod of Other renewables including bioenergy (TWh),Population density (people per sq. km of land area),"Gas (TWh, direct energy)","Oil (TWh, direct energy)",prod of Electricity from hydro (TWh),Fossil fuels (% growth),Annual change in primary energy consumption (%),Gas Consumption - TWh,Hydro Generation - TWh,prod of Electricity from wind (TWh),Oil Consumption - TWh,Entity_Category
1133105,Antarctica,1981,ATA,NaN,NaN,NaN,NaN,NaN,180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,336.450000,NaN,NaN,NaN,NaN,-0.273222,NaN,NaN,NaN,NaN,Continent
1133106,Antarctica,1981,ATA,NaN,NaN,NaN,NaN,NaN,14571611.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.062631,NaN,NaN,NaN,NaN,-0.273222,NaN,NaN,NaN,NaN,Continent
1133107,Antarctica,1981,ATA,NaN,NaN,NaN,NaN,NaN,652230.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.194838,NaN,NaN,NaN,NaN,-0.273222,NaN,NaN,NaN,NaN,Continent
1133108,Antarctica,1981,ATA,NaN,NaN,NaN,NaN,NaN,9045780.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.712523,NaN,NaN,NaN,NaN,-0.273222,NaN,NaN,NaN,NaN,Continent
1133109,Antarctica,1981,ATA,NaN,NaN,NaN,NaN,NaN,1246700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.930679,NaN,NaN,NaN,NaN,-0.273222,NaN,NaN,NaN,NaN,Continent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3429478,Antarctica,2019,ATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000072,NaN,NaN,NaN,NaN,Continent
3429479,Antarctica,2019,ATA,NaN,NaN,NaN,NaN,NaN,527970.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.234051,NaN,NaN,NaN,NaN,0.000072,NaN,NaN,NaN,NaN,Continent
3429480,Antarctica,2019,ATA,NaN,NaN,NaN,NaN,NaN,1213090.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.271989,NaN,NaN,NaN,NaN,0.000072,NaN,NaN,NaN,NaN,Continent
3429481,Antarctica,2019,ATA,NaN,NaN,NaN,NaN,NaN,743390.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.026465,NaN,NaN,NaN,NaN,0.000072,NaN,NaN,NaN,NaN,Continent


In [302]:
df_entity_lookup = df[['Entity_Category', 'Entity', 'Code', 'Income group', 'Lending category']].drop_duplicates().sort_values(by='Entity_Category')

In [306]:
df_entity_lookup['Entity_Category'].value_counts()

Country      233
Continent      1
World          1
Name: Entity_Category, dtype: int64

In [307]:
df_entity_lookup.head()

,Entity_Category,Entity,Code,Income group,Lending category
1133105,Continent,Antarctica,ATA,NaN,NaN
0,Country,United Kingdom,GBR,High income,NaN
6818,Country,Nepal,NPL,Lower middle income,IDA
6825,Country,Papua New Guinea,PNG,Lower middle income,Blend
6829,Country,Paraguay,PRY,Upper middle income,IBRD


In [305]:
df.head()

,Entity,Year,Code,Region,Income group,Lending category,Renewables (% electricity),"Coal (TWh, direct energy)",Land area (sq. km),Economy,Renewables per capita (kWh - equivalent),Fossil fuels (% equivalent primary energy),Fossil fuels per capita (kWh),Annual CO2 emissions,Geo Biomass Other - TWh,Coal Consumption - TWh,Wind Generation - TWh,prod of Electricity from solar (TWh),Per capita electricity (kWh),Coal Production - TWh,Gas Production - TWh,Fossil fuels (TWh),Solar Generation - TWh,Oil Production - TWh,prod of Other renewables including bioenergy (TWh),Population density (people per sq. km of land area),"Gas (TWh, direct energy)","Oil (TWh, direct energy)",prod of Electricity from hydro (TWh),Fossil fuels (% growth),Annual change in primary energy consumption (%),Gas Consumption - TWh,Hydro Generation - TWh,prod of Electricity from wind (TWh),Oil Consumption - TWh,Entity_Category
0,United Kingdom,1750,GBR,Europe & Central Asia,High income,NaN,NaN,NaN,NaN,United Kingdom,NaN,NaN,NaN,9350528.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Country
1,World,1750,OWID_WRL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9350528.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,World
2,United Kingdom,1751,GBR,Europe & Central Asia,High income,NaN,NaN,NaN,NaN,United Kingdom,NaN,NaN,NaN,9350528.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Country
3,World,1751,OWID_WRL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9350528.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,World
4,United Kingdom,1752,GBR,Europe & Central Asia,High income,NaN,NaN,NaN,NaN,United Kingdom,NaN,NaN,NaN,9354192.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Country


## Save Tables

In [39]:
df.head()

,Entity,Year,Code,Region,Income group,Lending category,Fossil fuels (% equivalent primary energy),Fossil fuels (% growth),Gas Production - TWh,Land area (sq. km),Unnamed: 0.1.1.1,Coal Consumption - TWh,Renewables per capita (kWh - equivalent),Wind Generation - TWh,prod of Other renewables including bioenergy (TWh),Fossil fuels per capita (kWh),Fossil fuels (TWh),Oil Production - TWh,Coal Production - TWh,Hydro Generation - TWh,Unnamed: 0.1.1,prod of Electricity from solar (TWh),prod of Electricity from hydro (TWh),Unnamed: 0.1,"Gas (TWh, direct energy)",Annual change in primary energy consumption (%),Geo Biomass Other - TWh,"Coal (TWh, direct energy)",Per capita electricity (kWh),Oil Consumption - TWh,Renewables (% electricity),Population density (people per sq. km of land area),Solar Generation - TWh,Gas Consumption - TWh,"Oil (TWh, direct energy)",Annual CO2 emissions,prod of Electricity from wind (TWh),Entity_Category
0,Afghanistan,1949,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14656.0,NaN,Country
1,Afghanistan,1950,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84272.0,NaN,Country
2,Afghanistan,1951,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91600.0,NaN,Country
3,Afghanistan,1952,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91600.0,NaN,Country
4,Afghanistan,1953,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106256.0,NaN,Country


In [40]:
df.to_csv('../data/silver_tables/yearly_values_per_entity.csv', index=False)

In [41]:
df_entity_lookup.to_csv('../data/silver_tables/entity_lookup.csv', index=False)